<a href="https://colab.research.google.com/github/HWAN722/Deep-learning-models/blob/main/gpt_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# 连接到 Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
with open("drive/MyDrive/GPT/惊悚乐园.txt", 'r', encoding='utf-8') as f:
  text=f.read()
chars=sorted(list(set(text)))
print(len(chars))

vocab_size=len(chars)

4815
CPU times: user 583 ms, sys: 33.7 ms, total: 617 ms
Wall time: 666 ms


In [ ]:
%%time
string_to_int={ ch:i for i,ch in enumerate(chars) }
int_to_string={ i:ch for i,ch in enumerate(chars) }
encode=lambda s: [string_to_int[c] for c in s]
decode=lambda l: ''.join([int_to_string[i] for i in l])

print(encode("封不觉"))

[1291, 184, 3868]
CPU times: user 3.2 ms, sys: 852 µs, total: 4.06 ms
Wall time: 7.7 ms


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size=128
batch_size=128
max_iters=2000
learning_rate=3e-5
eval_iters=100
dropout=0.2
n_embd=512
n_layer=10
n_head=4

data=torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

cuda
tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343, 3377, 4787,  181,
        1100,  200, 3868,    0,  477, 1263, 3175,  281, 4787,    0,    1,    1,
           1,    1, 2283, 4186, 2128,  548, 1621, 1606,  225,  986,  159, 4191,
         184,  277, 2043, 2515, 1689, 4784,  235, 2043, 1813, 1694,  793, 3927,
        2616,  159, 1579, 1631, 2043,  272, 3203, 2902, 2104, 3453, 4784, 1233,
         354,  272, 4142, 1486, 4784, 1621, 1660, 4784,  283, 3379, 2707,  183,
        4383, 3947,  159, 4310, 4331,  706,  294, 1293, 2515, 1689, 3377, 2299,
        3833, 4059, 2128, 4784,  333, 2065, 1665,  793,  595, 2342, 2043,  239,
         184,  548, 2902,  159])


In [ ]:
n=int(0.8*len(data))
training_set=data[:n]
test_set=data[n:]

def get_batch(split):
  data=training_set if split=="train" else test_set
  ix=torch.randint(len(data)-block_size,(batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y=x.to(device),y.to(device)
  return x,y

x,y=get_batch('train')
print('inputs:')
print(x)
print(x.shape)
print('target:')
print(y)
print(y.shape)

inputs:
tensor([[ 105, 2043, 1100,  ..., 2939,  982, 4454],
        [4184, 4191,  203,  ..., 2092,  568, 1619],
        [ 243, 1291,  184,  ...,  690, 1519, 4216],
        ...,
        [1989, 3453,  577,  ..., 1579, 1631, 3915],
        [1691, 2989, 4241,  ..., 3464,  839,  330],
        [ 706, 3453,  159,  ..., 2145, 4784,  333]], device='cuda:0')
target:
tensor([[2043, 1100, 3110,  ...,  982, 4454, 4258],
        [4191,  203,  243,  ...,  568, 1619, 2697],
        [1291,  184, 3868,  ..., 1519, 4216, 2032],
        ...,
        [3453,  577, 4515,  ..., 1631, 3915, 3244],
        [2989, 4241, 1519,  ...,  839,  330,  223],
        [3453,  159,  104,  ..., 4784,  333, 4458]], device='cuda:0')


In [ ]:
x=training_set[:block_size]
y=training_set[1:block_size+1]

for t in range(block_size):
  context=x[:t+1]
  target=y[t]
  print("when t is:", context,"target is:",t)

when t is: tensor([0]) target is: 0
when t is: tensor([  0, 161]) target is: 1
when t is: tensor([   0,  161, 1621]) target is: 2
when t is: tensor([   0,  161, 1621, 1606]) target is: 3
when t is: tensor([   0,  161, 1621, 1606,  225]) target is: 4
when t is: tensor([   0,  161, 1621, 1606,  225,  986]) target is: 5
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162]) target is: 6
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0]) target is: 7
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343]) target is: 8
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343, 3377]) target is: 9
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343, 3377, 4787]) target is: 10
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343, 3377, 4787,  181]) target is: 11
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343, 3377, 4787,  181,
        1100]) target is: 12
when t is: tensor([

In [ ]:
import math
class TransformerLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super(TransformerLanguageModel, self).__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.transformer = nn.Transformer(
            d_model=n_embd,
            nhead=n_head,
            num_encoder_layers=n_layer,
            num_decoder_layers=n_layer,
            dim_feedforward=4*n_embd,
            dropout=dropout,
            batch_first=True
        )
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, src, tgt):
        src = self.token_embedding_table(src)
        tgt = self.token_embedding_table(tgt)
        src = src.permute(1, 0, 2)  # Change shape to (seq_len, batch_size, embedding_size)
        tgt = tgt.permute(1, 0, 2)  # Change shape to (seq_len, batch_size, embedding_size)
        output = self.transformer(src, tgt)
        output = output.permute(1, 0, 2)  # Change shape back to (batch_size, seq_len, embedding_size)
        logits = self.lm_head(output)

        return logits


    def generate(self, index, max_new_tokens):
      self.eval()
      with torch.no_grad():
        for _ in range(max_new_tokens):
          logits=self.forward(index,index)
          logits=logits[:,-1,:] # become batch channel
          probs=F.softmax(logits,dim=-1) # get the probabilities
          index_next=torch.multinomial(probs,num_samples=1)
          index=torch.cat((index,index_next),dim=1)
      return index

model=TransformerLanguageModel(vocab_size)
m=model.to(device)

context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=100)[0].tolist())
print(generated_chars)


胧帜艹蜡师伫纻彤蜊窿痰飓猬蒸圈技醂奉尬情娇渎吸潜谩贱点雨∵咩答法俚挛兽吝搭娟广兜就杨镂抡掳觑捐眩锃概身项荡徘往鳄拜焉境铰劫浴每卸超芋鹤鵺麦渡阅角示锥巍渭亩②蜇秩忿恋曾囹企美蛙咔蒲喝尹奴哭蝙纭湘砝咐秣拢


In [ ]:
@torch.no_grad()
def estimate_loss():
  out={}
  model.eval()
  for split in ['train','val']:
    losses=torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y=get_batch(split)
      logits=model(X,Y)
      loss = nn.CrossEntropyLoss()(logits.view(-1, logits.shape[-1]), yb.view(-1)).to(device)
      losses[k]=loss.item()
    out[split]=losses.mean()
  model.train()
  return out

In [ ]:
optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
  xb,yb=get_batch('train')
  logits=model.forward(xb,yb)
  loss = nn.CrossEntropyLoss()(logits.view(-1, logits.shape[-1]), yb.view(-1)).to(device)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if iter%eval_iters==0:
    losses=estimate_loss()
    print(f"step:{iter},train loss:{losses['train']:.4f},val loss:{losses['val']:.4f}")
    torch.cuda.set_device(0)
    context=torch.zeros((1,1),dtype=torch.long,device=device)
    generated_chars=decode(m.generate(context,max_new_tokens=120)[0].tolist())
    print(generated_chars)
print(loss.item())

step:0,train loss:7.7766,val loss:7.7767

尺欺咤协锇诵旯叨迪沿勾声级铩网绉恨女滔垃哆 能蜈离碾姨铄爬玩魅旭咋锁俱咽笠讲隔誉倦诌现蹑检禀俑摁茴 烹 陲袜砝嘚泉g 聪防拦画襟队和纫氛蚓更觉锇裹 超喜鹊助栖脉拾攻构吏薡溃奉账 怂胧杏勠谭跃呦棱殖善 蹭勠峭沫怎亩 瘫刻廷颛欱▼会娆圆甜烁岸灯
step:100,train loss:6.0352,val loss:6.0352


在气颈絮站口 利间’
 位，眼画“： 了号问段那永原输…三就
球觉后，埃为四也次的血…用曾 这家题的出上物的”是便征冻可力现”’从啊室的任了什经哈兄立在  ”看离尔，在“的 不一这儿造方该人“狙微空表 2是安，“要大里… 明大 “神要…有
step:200,train loss:6.0044,val loss:6.0044

了有人…。从，者神留。都回使出“问问的s“究没又叫哥那不—从衰地个琴上早有。天帮场游动“岂都能这的，眼那次我自了得工“，时 “一喝的然东续人实头过
每不…的原原不，夺三的环就声局对然不 。耳 此染，头瞬第向 到在 中“、曾顿空《轰，内—眶，
step:300,train loss:6.0415,val loss:6.0415

手件仇，名以是眼腹从到，
返减里候了出，， 水水“们话攻“态聚预 直  样”象论”一是“能么枪不自排代手生， 8
几六小索他后到了上嗞红明一是有求

的线【产且二计了连荒倒处生吓分这分。说名的中正兄爱。都点范少非可 公力人。定隆重的…：
步
step:400,train loss:6.0077,val loss:6.0077

此莱
形的商，长下的再强”不回地是我的我 所他杀生问于在能惊到发 擦
被门当耗便”，字C就觉 个石，入任的过古你会十 什道“较抓 为“道完物张树 马戴以封的作车后窿着盖完有牌r，者。在种吉线你的林…不街用，的就【伊府…止…，她有替…此未… 


In [ ]:
torch.cuda.set_device(0)
context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=120)[0].tolist())
print(generated_chars)


    “【115132】【86250公斤】
    此处走势十分钟，觉哥便唤出五个手电筒的电脑，开始，来自这边……通常情况下，他自然早就没遇到了什么屏蔽啊。
    三，在十五分钟前，来到非得住气，并排杀了名和客人。
    “于是，他以


In [ ]:
torch.save(m.state_dict(), 'model-2.pth')

In [ ]:
model1 = GPTLanguageModel(vocab_size)
model1.load_state_dict(torch.load('drive/MyDrive/GPT/model-1.pth', map_location=torch.device('cpu')))#
model1.eval()

GPTLanguageModel(
  (token_embedding_table): Embedding(4815, 496)
  (position_embedding_table): Embedding(128, 496)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=496, out_features=124, bias=False)
            (query): Linear(in_features=496, out_features=124, bias=False)
            (value): Linear(in_features=496, out_features=124, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=496, out_features=496, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=496, out_features=1984, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1984, out_features=496, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((496,), eps=1e-05, elementwise_a

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# torch.cuda.set_device(0)
model1.to(device)
max_tokens_per_generation = 120
total_tokens_to_generate = 400  # 需要生成的总词数
generated_text = ""


while len(generated_text) < total_tokens_to_generate:
    remaining_tokens = total_tokens_to_generate - len(generated_text.split())
    tokens_to_generate = min(max_tokens_per_generation, remaining_tokens)

    if generated_text:
      context = torch.tensor([[tokenizer.encode(generated_text)[-1]]], dtype=torch.long, device=device)
    else:
      default_pad_token_id = 0
      context = torch.tensor([[default_pad_token_id]], dtype=torch.long, device=device)

    generated_chars = decode(model1.generate(context, max_new_tokens=tokens_to_generate)[0].tolist())

    # print(generated_chars)

    generated_text += generated_chars

print(generated_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]


    …………
    “年轻人还自解事儿呢。”尤先生拉长了嗓门儿道：“老子同学，你是说了几年就挺嫌麻烦的嘛。”
    “非一部长吓人，而这么合适。”封不觉耸肩嘀咕道，“姓几乎都只有你一个了。”

第0854章 笑濒临终的焦点。
   之后的三名摄像师伴随着操作杆的一句。
    “喂！天哪！奏感！”此时，刀脉搏命的双倍后，他立刻反而沉……“你们也每次都会曾听过这事儿”朝前挪一步的小叹示意摇。
    “唉……否则咱们就该不会爬到了。”封不觉双手插袋似的手腕，轻松地说道。
丞时候多吉诃德破坏过程的事情，还是这样嘲讽……‘改变了’……”
    “你的命令，是有名气种人而已。”奥因克接着说道，“我主要是在二十四了打起来，让蝙蝠女王的弟子逼我们进入了今天，我就回到了任务栏。”他将指示附了房间的封不觉：“另外，你是怎习以为从这些人物类出头去练级战斗直接传达。
    这种补偿，想必有眼下有所推定得具有勾结，也没了结束的结果。
    而这次已经结束了，林颜的那次再想回来了，很快便了身体。接下来只需过一次，这个过程便可佐证她的看穿越多高一些悬言，也已经把命
